# Fase 1 - Comprensión del negocio

En esta fase se identificarán los factores clave necesarios para abordar los objetivos empresariales y se plantearán en objetivos específicos de minería de datos, asegurando que el análisis posterior responda adecuadamente a las necesidades del negocio.

# Fase 2 - Comprensión de los datos

Durante esta fase, se realizará una recolección y análisis preliminar de los datos disponibles, con el objetivo de comprender su naturaleza y calidad. Se identificarán patrones iniciales y posibles problemas que puedan afectar el análisis, lo que permitirá guiar el proceso de preparación de los datos.

## Importaciones

In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import matplotlib.pyplot as plt

2025-06-28 17:07:47.087081: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-28 17:07:47.136439: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-28 17:07:47.136479: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-28 17:07:47.137906: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-28 17:07:47.145991: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-28 17:07:47.147037: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
# Verificación y configuración de GPU
print("GPU disponible: ", tf.config.list_physical_devices('GPU'))

# Habilitamos mixed precision para acelerar el entrenamiento en GPU
try:
    policy = tf.keras.mixed_precision.Policy('mixed_float16')
    tf.keras.mixed_precision.set_global_policy(policy)
    print('Mixed precision activada: ', policy.name)
except:
    print('Mixed precision no soportada en este entorno')

# Activar compilación XLA para acelerar el entrenamiento
tf.config.optimizer.set_jit(True)
print('Compilación XLA activada')

GPU disponible:  []
The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once
Mixed precision activada:  mixed_float16
Compilación XLA activada


## Descarga y lectura del texto

Se descarga el texto, se abre el archivo como binario, se lee y se decodifica en formato UTF-8, este es transformado en minúsculas.  

In [2]:
path = tf.keras.utils.get_file('quijote.txt', 'https://www.gutenberg.org/files/2000/2000-0.txt')
text = open(path, 'rb').read().decode(encoding='utf-8').lower()

2226045/2226045 [==============================] - 1s 0us/step


## Extracción de vocabulario único

Se crea un conjunto con los caracteres distintos, estos se ordenan alfabéticamente y se muestra la cantidad de caracteres únicos.

In [3]:
vocab = sorted(set(text))
print(f'{len(vocab)} caracteres únicos')

74 caracteres únicos


*Pasar a Codificación del texto a numéricos [Fase 3]*

# Fase 3 - Preparación de los datos

En esta fase se seleccionarán, limpiarán y transformarán los datos para que sean adecuados al modelado. Se abordarán problemas como valores faltantes, duplicados o inconsistencias, y se construirán nuevas variables cuando sea necesario, con el objetivo de obtener un conjunto de datos listo para aplicar las técnicas de modelado.

## Codificación del texto a numéricos

Se crea un diccionario que asigna a cada carácter único a un número entero, luego se crea un diccionario que asigna a cada carácter único a un número entero y se le asigna a cada carácter del texto su correspondiente número entero usando el diccionario.

In [4]:
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

## Creación de secuencias para el entrenamiento

Se divide el texto codificado en secuencias de 101 caracteres, los primeros 100 son usados como entrada y los siguientes 100 como objetivo, donde se prepararán así los datos en pares para entrenar una el modelo y este pueda aprender a predecir el siguiente carácter.

In [5]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

## Preparación de los batches

Se organiza el dataset de secuencias para el entrenamiento, donde se barajan aleatoriamente los datos con **BUFFER_SIZE = 10000** para evitar que el modelo aprenda el orden natural del texto, luego se agrupan las secuencias en lotes de 64 para garantizar que todos los batches tengan el mismo tamaño se usará drop_remainder=True.

In [ ]:
BATCH_SIZE = 128  # En lugar de 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

*Pasar a Construcción del modelo LSTM [Fase 4]*

# Fase 4 - Modelado

En esta fase se seleccionarán los algoritmos de modelado más adecuados y se entrenarán los modelos utilizando el conjunto de datos preparado. Se ajustarán los parámetros de los modelos y se evaluarán los resultados preliminares para determinar qué técnicas ofrecen el mejor desempeño para cumplir con los objetivos planteados.

## Construcción del modelo LSTM

Se define y construye el modelo LSTM, donde se definiran los parámetros:
* vocab_size: El tamaño del vocabulario.
* embedding_dim: La dimensión del espacio de representación vectorial para cada carácter.
* rnn_units: El número de unidades en la capa LSTM.

In [ ]:
vocab_size = len(vocab)
embedding_dim = 128  # Reducido de 256
rnn_units = 256      # Reducido de 1024

Se define la función la cual devuelve un modelo secuencial de Keras compuesto por una capa de entrada, la cual es necesaria para modelos stateful, una capa de Embedding, la cual transforma índices de caracteres en vectores densos, una capa LSTM stateful, con la que se generará una predicción por carácter, una capa Dropout del 25% para reducir el sobreajuste, una capa Dense final que entrega la probabilidad de cada carácter posible en la salida y se instancia el modelo con el batch size.

* *Capa de entrada: Se especifica explicitamente el tamaño del batch y de la secuencia esperada por el modelo, esto ya que corresponde a un modelo stateful, donde se requiere conocer de antemano el tamaño del batch porque el estado de la LSTM se mantiene entre batches.*
* *Capa de Embedding: Es una capa de proyección que toma índices enteros y los transforma en vectores densos de números realesw, esto debido a que el embedding aprende una representación vectorial útil de cada carácter.*
* *Capa LSTM: Esta es el núcleo del modelo, la LSTM procesará la secuencia de vectores y generará una salida por cada paso de tiempo.*
* *Dropout: Esta técnica de regularización desactiva aleatoriamente un porcentaje de neuronas durante el entrenamiento, lo cual evita que el modelo se sobreajuste a los datos de entrenamiento y ayuda a generalizar mejor.*
* *Capa Dense: Esta capa densa toma la salida de la LSTM y genera un vector de probabilidades sobre todos los caracteres posibles del vocabulario, esta tiene una salida la cual cuenta con un tamaño igual al número de caracteres únicos.*

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    return tf.keras.Sequential([
        tf.keras.layers.Input(batch_shape=(batch_size, None)),
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        # Usamos LSTM con mejor inicialización y optimizada para GPU
        tf.keras.layers.LSTM(rnn_units, 
                          return_sequences=True, 
                          stateful=True, 
                          recurrent_initializer='glorot_uniform',
                          recurrent_activation='sigmoid',  # Más eficiente que 'tanh'
                          implementation=2),  # Implementación más optimizada
        tf.keras.layers.Dropout(0.2),  # Reducido de 0.25
        tf.keras.layers.Dense(vocab_size)
    ])

model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

## Compilación del modelo

Se define una función de pérdida personalizada adecuada para clasificación multi-clase con etiquetas enteras, luego se compila utilizando el optimizador Adam, la función de pérdida y la métrica de precisión.

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Learning rate schedule para convergencia más rápida
initial_learning_rate = 0.002  # Un poco más alto para aprender más rápido inicialmente
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.9,
    staircase=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=[
        tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")
    ]
)

## Separación de datos en entrenamiento y validación

Se calcula el número total de batches y se divide el dataset en un 80%-20% para el entrenamiento y la validación respectivamente, buscando evaluar el modelo con datos no vistos en su entrenamiento.

In [ ]:
total_batches = dataset.cardinality().numpy()
train_size = int(0.8 * total_batches)

train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(AUTOTUNE)
val_dataset = val_dataset.prefetch(AUTOTUNE)


NameError: name 'dataset' is not defined

## Entrenamiento del modelo con los callbacks

Se entrena el modelo por 20 épocas con dos callbacks importantes ModelCheckpoint, el cual guarda los pesos del modelo por época, lo que permite retomar el entrenamiento si se interrumpe y conserva el mejor modelo EarlyStopping, el cual detiene el entrenamiento si la precisión en validación no mejora tras 2 épocas, restaurando automáticamente los mejores pesos. Luego se entrena con los datasets ya preparados y separados (entrenamiento y validación) y guarda el historial de métricas para análisis posterior.

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
    save_freq=5 * len(train_dataset)  # Guardar cada 5 épocas en lugar de cada época
)

earlystop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=3,  # Aumentado para dar más tiempo para alcanzar el objetivo
    restore_best_weights=True
)

# Agregar TensorBoard para monitoreo de rendimiento
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='./logs',
    histogram_freq=1,
    profile_batch='500,520'  # Perfilar algunos batches para analizar rendimiento
)

EPOCHS = 20
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=[checkpoint_callback, earlystop_callback, tensorboard_callback]
)

Epoch 1/20
186/268 [===================>..........] - ETA: 2:17 - loss: 2.3940 - accuracy: 0.3032

KeyboardInterrupt: 

*Se pasa a Visualización de las métricas de entrenamiento y validación [Fase 5]*

# Fase 5 - Evaluación

En la fase de evaluación se analizará el rendimiento de los modelos desarrollados, comparándolos con los objetivos del negocio para asegurar que sean útiles y precisos. Se tomará en cuenta la validez de los resultados, se decidirá si es necesario ajustar los modelos o si están listos para su implementación.

## Visualización de las métricas de entrenamiento y validación

Se analiza el rendimiento del modelo LSTM visualizando gráficamente la evolución de la pérdida y la precisión durante las épocas de entrenamiento y validación.

In [ ]:
def plot_training_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Entrenamiento')
    plt.plot(epochs_range, val_acc, label='Validación')
    plt.title('Precisión durante el entrenamiento')
    plt.xlabel('Épocas')
    plt.ylabel('Precisión')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Entrenamiento')
    plt.plot(epochs_range, val_loss, label='Validación')
    plt.title('Pérdida durante el entrenamiento')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_training_history(history)

## Generación de una palabra

Se crea una función que genera una palabra de forma carácter por carácter, donde se transforma la entrada en índices, se reinicia el estado de la LSTM, y en cada paso predice el siguiente carácter.

In [ ]:
def generate_word(model, start_char, temperature=1.0, max_chars=30):
    input_eval = [char2idx[c] for c in start_char.lower() if c in char2idx]
    input_eval = tf.expand_dims(input_eval, 0)
    model.layers[1].reset_states()

    word = start_char
    for _ in range(max_chars):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0) / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        next_char = idx2char[predicted_id]
        word += next_char

        if next_char == ' ':
            break
        input_eval = tf.expand_dims([predicted_id], 0)

    return word.strip()

## Generación de una frase

Se crea una función que genera una frase completa construyendo palabra por palabra en un rango definido (temperature indica que tan creativo o tradicional será lo generado).

In [ ]:
def generate_phrase(model, start_word, temperature=1.0, max_words=10):
    current_input = start_word
    phrase = start_word.strip()

    for _ in range(max_words - 1):
        next_word = generate_word(model, current_input[-1], temperature=temperature)
        phrase += ' ' + next_word
        current_input = next_word

    return phrase

## Evaluación de la generación de palabras y frases

* La función generate_word(model, 'a') genera una palabra comenzando desde la letra 'a'.

* La función generate_word(model, palabra_inicial[-1]) genera una palabra a partir del último carácter de la palabra "don".

* La función generate_phrase(model, "caballo", temperature=0.8): genera una frase entera palabra por palabra, empezando con la palabra "caballo".

In [ ]:
print("Palabra generada desde letra 'a':", generate_word(model, 'a'))

palabra_inicial = "don"
nueva_palabra = generate_word(model, palabra_inicial[-1])
print(f"Palabra generada desde la última letra de '{palabra_inicial}':", nueva_palabra)

print("Frase generada desde 'caballo':")
print(generate_phrase(model, "caballo", temperature=0.8))

# Fase 6 - Implementación

En esta fase los modelos serán implementados en un entorno productivo para su uso real. Se generarán reportes detallados sobre los resultados y se establecerá un plan de mantenimiento y actualización periódica para asegurar la continua eficacia del modelo en el tiempo.